In [ ]:
import requests
import unicodedata
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from lxml import etree
from bs4 import BeautifulSoup as soup

In [ ]:
def zdb_sru(query):
    base_url = "http://services.dnb.de/sru/zdb"
    parameter = {"recordSchema" : "MARC21plus-1-xml",
                 "operation" : "searchRetrieve",
                 "version" : "1.1",
                 "maximumRecords" : "100",
                 "query" : query}

    req = requests.get(base_url, params=parameter)
    print(req.url)
    xml = soup(req.content, features="xml")

    records = xml.find_all("record", {"type" : "Bibliographic"})

    if len(records) < 100:
        return records
    else:
        results = 100
        i = 101
        while results == 100:
            parameter.update({"startRecord" : i})
            req = requests.get(base_url, params=parameter)
            xml = soup(req.content, features="lxml")
            new_records = xml.find_all("record", {"type" : "Bibliographic"})
            records = new_records
            i += 100
            results = len(new_records)

    return records

In [ ]:
def laufzeit_parser(laufzeit):    
    if laufzeit == []:
        laufzeit = [""]
        
    runtime_list = laufzeit[0].replace(" ", "")
    runtime_list = list(runtime_list.split(";"))

    total_runtime = []
    for i in runtime_list:
        reg = re.findall(r"1[8-9][0-9]{2}", i)
        

        if len(reg) > 1:
            fill_arr = np.arange(int(reg[0]), int(reg[1])+1, 1)
            for i in fill_arr:
                total_runtime.append(i)
        elif reg:
            total_runtime.append(reg[0])

    #Umwandlung der Jahrgänge in Liste in Integer, Eliminierung von Duplikaten
    bereinigt = list(map(int, total_runtime))     
    bereinigt = list(set(bereinigt))    
    bereinigt.sort()    

    return bereinigt

In [ ]:
def parse_record(record):
    namespace = {"marc" : "http://www.loc.gov/MARC21/slim"}
    xml = etree.fromstring(unicodedata.normalize("NFC", str(record)))
    
    data_list = []

    zeitraum = xml.xpath("marc:datafield[@tag = '363']/marc:subfield[@code = 'i']", namespaces=namespace)        
    
    try:
        zeitraum = zeitraum[0].text
        zeitraum = int(zeitraum[:4])
    except:
        zeitraum = 0
    
    if not zeitraum >= 1945:    
    
        for i in xml.findall("marc:datafield[@tag='924']", namespaces=namespace):

            #IDN
            idn = xml.xpath("marc:controlfield[@tag = '001']", namespaces=namespace)

            try:
                idn = idn[0].text
            except:
                idn = "N.N."

            #Titel        
            titel = xml.xpath("marc:datafield[@tag = '245']/marc:subfield[@code = 'a']", namespaces=namespace)

            try:
                titel = titel[0].text
            except:
                titel = "N.N."

            #Erscheinungsort
            ort = xml.xpath("marc:datafield[@tag = '264']/marc:subfield[@code = 'a']", namespaces=namespace)

            try:
                ort = [angabe.text for angabe in ort]
            except:
                ort = "N.N."

            #Erscheinungsfrequenz
            frequenz = xml.xpath("marc:datafield[@tag = '515']/marc:subfield[@code = 'a']", namespaces=namespace)

            try:
                frequenz = frequenz[0].text
            except:
                frequenz = "N.N."

            #Erscheinungsverlauf
            verlauf = xml.xpath("marc:datafield[@tag = '362']/marc:subfield[@code = 'a']", namespaces=namespace)

            try:
                verlauf = verlauf[0].text
            except:
                verlauf = "N.N."

            #Besitzende Einrichtung        
            einrichtung = i.find("marc:subfield[@code='b']", namespaces=namespace)
            einrichtung = einrichtung.text

            #Materialart
            material = xml.xpath("marc:datafield[@tag = '337']/marc:subfield[@code = 'a']", namespaces=namespace)

            try:
                material = [mat.text for mat in material]
            except:
                material = "N.N."

            #Signatur
            signatur = i.find("marc:subfield[@code = 'g']", namespaces=namespace)

            try:
                signatur = signatur.text
            except:
                signatur = "N.N."

            #Bestand der Einrichtung
            zeit = i.findall("marc:subfield[@code='z']", namespaces=namespace)

            try:
                bestand = [item.text for item in zeit if len(zeit) != 0]

                if bestand == []:
                    zeit1 = i.findall("marc:subfield[@code='q']", namespaces=namespace)
                    zeit2 = i.findall("marc:subfield[@code='v']", namespaces=namespace)

                    bestand1 = [item.text for item in zeit1]
                    bestand2 = [item.text for item in zeit2]

                    bestand = bestand1 + bestand2                    
                    bestand = ["-".join(bestand)]

            except:
                bestand = [""]

            data_list.append([idn, titel, ort, frequenz, verlauf, einrichtung, signatur, bestand, material])
            
    return data_list

In [ ]:
def parse_record(record):
    namespace = {"marc" : "http://www.loc.gov/MARC21/slim"}
    xml = etree.fromstring(unicodedata.normalize("NFC", str(record)))
    
    data_list = []

    zeitraum = xml.xpath("marc:datafield[@tag = '363']/marc:subfield[@code = 'i']", namespaces=namespace)        
    
    try:
        zeitraum = zeitraum[0].text
        zeitraum = int(zeitraum[:4])
    except:
        zeitraum = 0
    
    if not zeitraum >= 1945:    
    
        for i in xml.findall("marc:datafield[@tag='924']", namespaces=namespace):

            #IDN
            idn = xml.xpath("marc:controlfield[@tag = '001']", namespaces=namespace)

            try:
                idn = idn[0].text
            except:
                idn = "N.N."

            #Titel        
            titel = xml.xpath("marc:datafield[@tag = '245']/marc:subfield[@code = 'a']", namespaces=namespace)

            try:
                titel = titel[0].text
            except:
                titel = "N.N."

            #Erscheinungsort
            ort = xml.xpath("marc:datafield[@tag = '264']/marc:subfield[@code = 'a']", namespaces=namespace)

            try:
                ort = [angabe.text for angabe in ort]
            except:
                ort = "N.N."

            #Erscheinungsfrequenz
            frequenz = xml.xpath("marc:datafield[@tag = '515']/marc:subfield[@code = 'a']", namespaces=namespace)

            try:
                frequenz = frequenz[0].text
            except:
                frequenz = "N.N."

            #Erscheinungsverlauf
            verlauf = xml.xpath("marc:datafield[@tag = '362']/marc:subfield[@code = 'a']", namespaces=namespace)

            try:
                verlauf = verlauf[0].text
            except:
                verlauf = "N.N."

            #Besitzende Einrichtung        
            einrichtung = i.find("marc:subfield[@code='b']", namespaces=namespace)
            einrichtung = einrichtung.text

            #Materialart
            material = xml.xpath("marc:datafield[@tag = '337']/marc:subfield[@code = 'a']", namespaces=namespace)

            try:
                material = [mat.text for mat in material]
            except:
                material = "N.N."

            #Signatur
            signatur = i.find("marc:subfield[@code = 'g']", namespaces=namespace)

            try:
                signatur = signatur.text
            except:
                signatur = "N.N."

            #Bestand der Einrichtung
            zeit = i.findall("marc:subfield[@code='z']", namespaces=namespace)

            try:
                bestand = [item.text for item in zeit if len(zeit) != 0]

                if bestand == []:
                    zeit1 = i.findall("marc:subfield[@code='q']", namespaces=namespace)
                    zeit2 = i.findall("marc:subfield[@code='v']", namespaces=namespace)

                    bestand1 = [item.text for item in zeit1]
                    bestand2 = [item.text for item in zeit2]

                    bestand = bestand1 + bestand2                    
                    bestand = ["-".join(bestand)]

            except:
                bestand = [""]

            laufzeit_liste = laufzeit_parser(bestand)

            for jahr in laufzeit_liste:
                if jahr >= 1850:
                    data_list.append([idn, titel, ort, frequenz, verlauf, einrichtung, signatur, jahr, material])
            
    return data_list

In [ ]:
def to_df(ergebnis):
    df_list = []
    columns = ["IDN", "Titel", "Erscheinungsort", "Erscheinungsfrequenz", 
               "Verlauf", "Einrichtung", "Signatur", "Bestand", "Material"]

    for element in ergebnis:    
        df = pd.DataFrame(element, columns=columns)    
        df_list.append(df)

    df_origin = pd.concat(df_list, ignore_index=True)

    return df_origin

In [ ]:
ortsliste = ["Hannover"]
df_liste = []

for ort in ortsliste:
    records = zdb_sru("dok=zeitung and vort={}".format(ort))
    ergebnis = [parse_record(record) for record in records]
    
    if ergebnis:
        df_interim = to_df(ergebnis)
        df_liste.append(df_interim)

In [ ]:
main_df = pd.concat(df_liste, ignore_index=True)

#main_df.to_excel("sru_abfrage.xlsx")
#main_df.to_csv("sru_abfrage.csv")

print(main_df)

In [ ]:
def unikalcheck(df):
        df_dedup = df.drop_duplicates(subset=["Titel", "Bestand"], keep=False)
        
        df_unikal = df_dedup[df_dedup["Einrichtung"] == "DE-3"]
                        
        #df_unikal.to_csv("unikal.csv")
        
        return df_unikal

In [ ]:
def zeitungsplot(df, plot_titel):
    df = df[df["Einrichtung"] == "DE-3"]
    
    jahrgangsplot = df["Bestand"].value_counts()

    a = [x for x in range(1850, 1990,1)]
    b = []
    
    for i in a:
        if i not in jahrgangsplot.index:
            b.append(i)

    filler = pd.Series(0, index=b)
    jahrgangsplot = pd.concat([jahrgangsplot, filler])
    jahrgangsplot = jahrgangsplot.sort_index()
    jahrgangsplot.index = jahrgangsplot.index.astype("int")

    plt.figure(figsize=(13,10)) 
    barplot = jahrgangsplot.plot.bar()

    #Beschriftung der X-Achse in Abständen von fünf Jahren
    for i, j in enumerate(barplot.get_xticklabels()):
        if (i % 5) != 0:        
            j.set_visible(False)

    plt.title(plot_titel)
    plt.xlabel("Jahrgang")
    plt.ylabel("Menge")
    
    #plt.savefig("Plot_Zeitungsbestand.png")
    #plt.savefig("Plot_Zeitungsbestand.pdf") 
    #plt.savefig("Plot_Zeitungsbestand.svg") 
    #plt.savefig("Plot_Zeitungsbestand.tiff")

In [ ]:
df_unikal = unikalcheck(main_df)

zeitungsplot(main_df, "Zeitungsbestand (gesamt)")
zeitungsplot(df_unikal, "Zeitungsbestand (unikal)")